### How to use Jupyter Notebook:

Jupyter Notebook is split into cell blocks and can run a script one cell block at a time. 

To run the cells with code, click the cell then click "Run" on the toolbar above or hit Ctrl+Enter on the keyboard. Runable cells have a gray background and In[ ] on the left side.

See the general process of the code and common scripts in the "README.md" file and "CBECC-com python basics" Jupyter Notebook.

# <span style="color:#1579F3">Hardcoding Autosized VAV Box Capacities.</span>

Because CBECC-com does not allow for autosized values to be used for compliance, the user must enter hardsized values for each equipment capacity in order to produce valid compliance documents or else a "Not usable for Compliance" watermark will appear. This script can extract and replace autosized values for all VAV boxes in the CBECC-com model and removes the watermark. 

*This is meant to help speed up the replacement of autosize values for early design or Core+Shell projects. Use real hardsized values when available*

<img src="Images/ComplianceWatermark.png" width="400" align="left"></img>

### Important Notes:
- Save this Jupyter Notebook file in the same folder as the CBECC-com file (.cibdx, cibd16x, cibd19x, or .xml). It is recommended to keep a backup file.
- This script pulls hard and autosized values and cannot overwrite hardsized with autosized values. Hard-sized values can be removed using a simple script. See "Remove VAV reheat coil capacity.ipynb" as an example.
- Does not work for active chilled beams or fan-powered boxes.
- Currently cannot update the Assigned AHU. Change the Assigned AHU, click and drag in CBECC-com.
- This script does not check for CBECC-com errors so make sure your excel sheet inputs are appropriate.

## <span style='color:#1579F3'>Step 1: Setting up the CBECC-com file to autosize values</span>
The autosized values of CBECC-com are based on rough rule of thumb equations and not from the EnergyPlus run. The default is 1.00 cfm/sf and 400 cfm/ton. *These autosized values do not guarentee no unmet hours.*

### Turn on autosizing
1. It is recommended to create a save as copy of the original CBECC-com file to extract autosized values.
2. In the CBECC-com tree, double click "Projects" and open the "Building Model Data" dialogue.
3. Go to the "Analysis Option" tab.
4. Check on "Auto-Populate Proposed HVAC Capacities".
5. Change the "Design" on the right as necessary to change rule of thumb assumptions.
6. Check the terminal unit or the reheat coil you want to autosize and make sure the values are fill out in blue text
7. Save the project.
8. <span style='color:red'>Close then reopen the project to update the "*filename* - post open.xml" file. </span> The script does not work without the "post open" file updated.

## <span style='color:#1579F3'>Step 2: Setting up the script.</span>
Enter the file name of the CBECC-com model you would like to modify and run the following cell block (Ctrl+Enter).

In [ ]:
# Enter the file name of the CBECC-com model. File paths or links do not work.
file = ('SRJC STEM Building Title 24_noVAV.cibd16x')

from lxml import etree as ET
import pandas as pd

filename = file.split('.')[0]
filetype = file.split('.')[1]

detail_file = filename + ' - post open.xml'

tree = ET.parse(file)
root = tree.getroot()

dtree = ET.parse(detail_file)
droot = dtree.getroot()

excel_file = filename + ' - dVAVData.xlsx'

## <span style='color:#1579F3'>Step 3: Extracting autosized values</span>
Following script extracts the autosize information from the "post open" file into a dataframe then writes the dataframe into Excel.

The extracted excel file will be called "*filename* - dVAVData.xlsx". Make sure to close the Excel file if re-running Step 1.

Run the following cell block. No changes to the code is necessary.

In [ ]:
trml_unit_list = []

# Only for uncontrolled or VAV boxes. No fan-powered boxes or active beams
for unit in droot.iter('TrmlUnit'):
    trml = {}
    
    trml['Terminal Unit Name'] = unit.find('Name').text
    trml['Type'] = unit.find('Type').text
    trml['Assigned AHU'] = unit.getparent().find('Name').text
    trml['Zone Served'] = unit.find('ZnServedRef').text
    trml['Primary Flow'] = float(unit.find('PriAirFlowMax').text)
    try:
        trml['Primary Min. Flow'] = float(unit.find('PriAirFlowMin').text)
    except:
        pass
    
    if trml['Type'] == 'VAVReheatBox':
        try:
            trml['Heating Air Flow'] = float(unit.find('HtgAirFlowMax').text)
        except:
            pass
        try:
            trml['Reheat Coil Name'] = unit.find('CoilHtg/Name').text
        except:
            pass
        try:
            trml['Reheat Coil Type'] = unit.find('CoilHtg/Type').text
        except:
            pass
        if trml['Reheat Coil Type'] == 'HotWater':
            try:
                trml['Hot Water Inlet Segment'] = unit.find('CoilHtg/FluidSegInRef').text
            except:
                pass
            try:
                trml['Hot Water Outlet Segment'] = unit.find('CoilHtg/FluidSegOutRef').text
            except:
                pass
        
        heating_cap = 0
        try:
            heating_cap += float(unit.find('CoilHtg/CapTotNetRtd').text)
        except:
            try:
                heating_cap += float(unit.find('CoilHtg/CapTotGrossRtd').text)
            except:
                pass
        trml['Reheat Coil Capacity'] = heating_cap
    trml_unit_list.append(trml)
    
trml_header = [
    'Terminal Unit Name', 'Type', 'Assigned AHU', 'Zone Served', 
    'Primary Flow', 'Primary Min. Flow', 'Heating Air Flow', 
    'Reheat Coil Name', 'Reheat Coil Type', 'Hot Water Inlet Segment', 
    'Hot Water Outlet Segment', 'Reheat Coil Capacity'
]
    
trml_df = pd.DataFrame(trml_unit_list, columns=trml_header)

writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')
trml_df.to_excel(writer,sheet_name='Terminal Units')
writer.save()

## <span style='color:#1579F3'>Step 4: Import autosized values into CBECC-com</span>
Make sure the Excel file is closed and run the following script. 

The new CBECC-com file will be called "*filename*_new".

Notes:
- The Excel sheet can be modified and re-imported again without re-running Step 1.
- Make sure to check for any warnings and errors immediately after running the code block.
- No changes to the code is necessary.

In [ ]:
### Re-parsing root file in case Excel file is attempted to be re-imported

tree = ET.parse(file)
root = tree.getroot()



### Creating a new dataframe from the updated excel sheet and modifying the xml file. 

trml_df_new=pd.read_excel(excel_file,sheet_name='Terminal Units', index_col=0)

for i,row in trml_df_new.iterrows():
    unit = root.xpath('.//TrmlUnit/Name[text()="{}"]'.format(row['Terminal Unit Name']))[0].getparent()
    
    unit.find('Type').text = str(row['Type'])
    unit.find('ZnServedRef').text = str(row['Zone Served'])
    unit.find('PriAirFlowMax').text = str(row['Primary Flow'])
    try:
        unit.find('PriAirFlowMin').text = str(row['Primary Min. Flow'])
    except:
        if row['Type'] != 'Uncontrolled':
            min_flow = ET.SubElement(unit, 'PriAirFlowMin')
            min_flow.text = str(row['Primary Min. Flow'])
    if row['Type'] == 'VAVReheatBox':
        try:
            unit.find('HtgAirFlowMax').text = str(row['Heating Air Flow'])
        except:
            heat_flow = ET.SubElement(unit, 'HtgAirFlowMax')
            if str(row['Heating Air Flow']) != 'nan':
                heat_flow.text = str(row['Heating Air Flow'])
            else:
                display(Markdown('### <span style="color:orange">Warning - Heating airflow rate for terminal unit {} is blank</span>'.format(row['Terminal Unit Name'])))
                pass
        
        if unit.find('CoilHtg') == None:
            HtgCoil = ET.SubElement(unit, 'CoilHtg')
        
        CoilHtg = unit.find('CoilHtg')
        
        try:
            CoilHtg.find('Name').text = str(row['Reheat Coil Name'])
        except:
            rhc_name = ET.SubElement(CoilHtg, 'Name')
            if str(row['Reheat Coil Name']) != 'nan':
                rhc_name.text = str(row['Reheat Coil Name'])
            else:
                display(Markdown('### <span style="color:crimson">ERROR! - Enter name for heating coil in terminal unit {}</span>'.format(row['Terminal Unit Name'])))
                break
        try:
            CoilHtg.find('Type').text = str(row['Reheat Coil Type'])
        except:
            rhc_type = ET.SubElement(CoilHtg, 'Type')
            if str(row['Reheat Coil Type']) != 'nan':
                rhc_type.text = str(row['Reheat Coil Type'])
            else:
                display(Markdown('### <span style="color:orange">Warning - Enter reheat coil type for {}</span>'.format(rhc_name.text)))
                pass
        
            if rhc_type.text == 'HotWater':
                try: 
                    CoilHtg.find('FluidSegInRef').text = str(row['Hot Water Inlet Segment'])
                except:
                    rhc_hwi = ET.SubElement(CoilHtg, 'FluidSegInRef')
                    if str(row['Hot Water Inlet Segment']) != 'nan':
                        rhc_hwi.text = str(row['Hot Water Inlet Segment'])
                    else:
                        display(Markdown('### <span style="color:orange">Warning - Hot water inlet fluid segment for {} is blank</span>'.format(rhc_name.text)))
                        pass
                
                try: 
                    CoilHtg.find('FluidSegOutRef').text = str(row['Hot Water Outlet Segment'])
                except:
                    rhc_hwi = ET.SubElement(CoilHtg, 'FluidSegOutRef')
                    if str(row['Hot Water Outlet Segment']) != 'nan':
                        rhc_hwi.text = str(row['Hot Water Outlet Segment'])
                    else:
                        display(Markdown('### <span style="color:orange">Warning - Hot water outlet fluid segment for {} is blank</span>'.format(rhc_name.text)))
                        pass
        try:
            CoilHtg.find('CapTotGrossRtd').text = str(row['Reheat Coil Capacity'])
        except:
            rhc_cap = ET.SubElement(CoilHtg, 'CapTotGrossRtd')
            if str(row['Reheat Coil Capacity']) != 'nan':
                rhc_cap.text = str(row['Reheat Coil Capacity'])
            else:
                display(Markdown('### <span style="color:orange">Warning - Reheat coil capacity for {} is blank</span>'.format(rhc_name.text)))
                pass
                
tree.write(filename + '_new' + filetype, pretty_print=True)

## <span style='color:#1579F3'>Step 4: Check the newly created CBECC-com file and turn off autosizing</span>

Open the new CBECC-com file and make sure the program can properly read the updated inputs. 

Using the same process as Step 1, turn off the autosizing to remove the watermark.

Re-run the cell blocks as necessary. If you to re-run this entire script based on the new CBECC-com file, move/delete the original and rename the new file to the original file or change the file name input in Step 1.